<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/WienerPathTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Upgrade to TensorFlow nightly
!pip install --upgrade tf-nightly-cpu

     |████████████████████████████████| 398.8MB 36kB/s 
     |████████████████████████████████| 1.3MB 47.2MB/s 
     |████████████████████████████████| 471kB 49.5MB/s 
     |████████████████████████████████| 12.2MB 46.0MB/s 
     |████████████████████████████████| 3.8MB 43.1MB/s 
     |████████████████████████████████| 14.5MB 338kB/s 
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4
  Found existing installation: grpcio 1.33.2
    Uninstalling grpcio-1.33.2:
      Successfully uninstalled grpcio-1.33.2
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.1

In [1]:
import tensorflow as tf
from scipy.special import erf
import numpy as np
import time
import math

In [2]:
class UnivariateBrownianBridge():
  def __init__(self, number_time_steps):
    self.number_time_steps = number_time_steps

    self.left_index = np.zeros(number_time_steps, dtype=int)
    self.right_index = np.zeros(number_time_steps, dtype=int)
    self.bridge_index = np.zeros(number_time_steps, dtype=int)
    self.left_weight = np.zeros(number_time_steps)
    self.right_weight = np.zeros(number_time_steps)
    self.std_dev = np.zeros(number_time_steps)

    self._map = np.zeros(number_time_steps, dtype=int)

    self._map[-1] = 1
    self.bridge_index[0] = number_time_steps - 1
    self.std_dev[0] = math.sqrt(1.0 * number_time_steps)
    self.left_weight[0] = 0
    self.right_weight[0] = 0

    j=0
    for i in range(1,number_time_steps):
      while self._map[j] == True:
        j = j + 1
      k = j
      while self._map[k] == False:
        k = k + 1
      l = j+((k-1-j)>>1)
      self._map[l]=i
      self.bridge_index[i]=l
      self.left_index[i]=j
      self.right_index[i]=k
      self.left_weight[i]=(k-l)/(k+1-j)
      self.right_weight[i]=(1+l-j)/(k+1-j)
      self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
      j=k+1
      if j>=number_time_steps:
        j=0

    self.bridge_index_t = tf.constant(tf.convert_to_tensor(self.bridge_index,dtype=tf.int32))
    self.right_index_t = tf.constant(tf.convert_to_tensor(self.right_index,dtype=tf.int32))
    self.left_index_t = tf.constant(tf.convert_to_tensor(self.left_index,dtype=tf.int32))
    self.std_dev_t = tf.constant(tf.convert_to_tensor(self.std_dev))
    self.right_weight_t = tf.constant(tf.convert_to_tensor(self.right_weight))
    self.left_weight_t = tf.constant(tf.convert_to_tensor(self.left_weight))

  @tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64)))
  def buildPath(z, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path = tf.TensorArray(dtype=tf.float64,size=number_time_steps+1)
    path = path.write(number_time_steps,std_dev[0]*z[0]);
    path = path.write(0,0*z[0]);  
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      path = path.write(l+1,left_weight[i] * path.read(j) + right_weight[i] * path.read(k+1) + std_dev[i] * z[i])
    return path.stack()

  def path(self,z):
    return UnivariateBrownianBridge.buildPath(z, self.number_time_steps, self.left_index_t, self.right_index_t, self.bridge_index_t, self.left_weight_t, self.right_weight_t, self.std_dev_t)

In [3]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  4#@param {type:"integer"}
number =  10#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
z = tf.transpose(tf.math.erf(x))

brownian.path(z)

<tf.Tensor: shape=(5, 10), dtype=float64, numpy=
array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.88854887, 0.68913328, 0.99660398, 0.84481476, 0.8832936 ,
        0.93620334, 0.66753235, 0.85165759, 1.03729986, 0.91588149],
       [1.04099976, 0.98748202, 0.98748202, 0.80823382, 1.56815012,
        0.76355709, 0.76355709, 0.55058631, 1.41854869, 0.88553438],
       [1.40904875, 1.40028891, 1.27293037, 1.36265861, 1.85390393,
        1.44571728, 0.62131329, 0.81243635, 1.95472344, 1.40573812],
       [1.04099976, 1.42231127, 0.55265278, 0.80823382, 1.56815012,
        1.24648176, 0.28063241, 0.41823535, 1.33816933, 1.6302048 ]])>

In [21]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = brownian.path(z)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)

Runtime on CPU:  0.43997859954833984


In [15]:
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

time_start = time.time()

brownian = UnivariateBrownianBridge(dim)

with tf.profiler.experimental.Profile("logs/brownian"):
  with tf.device('/cpu'):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(tf.math.erf(x))
    w = brownian.path(z)
time_end = time.time()
time_price_tpu = time_end - time_start
print("Time on a CPU: ", time_price_tpu)    

Time on a CPU:  0.6087753772735596


In [16]:
!pip install -U tensorboard_plugin_profile

     |████████████████████████████████| 1.1MB 11.6MB/s 


In [17]:
%load_ext tensorboard